In [0]:

#this is the implemented Jaccard distance
def Jaccard_distance(feat_one, feat_two):
  union = 0
  intersection = 0

  string1 = ''
  string2 = ''

  stringArray1 = {}
  stringArray2 = {}

  j = 0
  for i in range(len(feat_one)):
    if feat_one[i] != ' ':
      string1+=feat_one[i]
    else :
      stringArray1[j] = string1
      j+=1
      string1 = ''

  j = 0
  for i in range(len(feat_two)):
    if feat_two[i] != ' ':
      string1+=feat_two[i]
    else :
      stringArray2[j] = string1
      j+=1
      string1 = ''

  for i in range(len(stringArray1)) :
    for j in range(len(stringArray2)) :
      if stringArray1[i] == stringArray2[j]:
        intersection+=1

  union = len(stringArray1) + len(stringArray2) - intersection
  distance = 1 - (intersection/union)

  return distance

In [0]:
# using classified elements, find the most commmon elements out of the classified, and send it back as a new centroid

import time
def fiveCommon(feat_one):

  commonArray = {}

  discoveredArray = {}

  

  string1 = ''
  j = 0
  center = feat_one[0]
  
  appearanceArray = [1]*len(commonArray)
  testingArray = {}
  allWords = {}

  wordAppearance = {}

  tempArray = {}

  j = 0
  
  k = 0

  #add stuff to dictionary, and then check for the top words that appear
  for x in range(0, len(feat_one)):
    sentence = feat_one[x]
    for i in range(len(sentence)):
      if sentence[i] != ' ':
        string1+=sentence[i]
      else:
        tempArray[j] = string1
        j+=1
        string1 = ''

    j = 0

    found = False

    for i in tempArray:
      if tempArray[i] in wordAppearance:
          wordAppearance[tempArray[i]] = wordAppearance[tempArray[i]] + 1
      else:
        wordAppearance[tempArray[i]] = 1
  
  count1 = 0

  #we create a dictionary out of all the words in the classification, and then take the top
  #5 words that appear in the classification
  for i in wordAppearance:
    if count1 < 5:
      commonArray[i] = wordAppearance[i]
      count1+=1
    else:
      smallest = ''
      count2 = 0
      for k in commonArray:
        if count2 < 1:
          smallest = k
          count2 = 1
        elif commonArray[k] < commonArray[smallest]:
          smallest = k
      if commonArray[smallest] < wordAppearance[i]:
        del commonArray[smallest]
        commonArray[i] = wordAppearance[i]
  count = 0
  for i in commonArray:
    discoveredArray[count] = i
    count+=1
  
  return discoveredArray

In [0]:
import numpy as np
import pandas as pd 
import time

class K_Means:
	def __init__(self, k = 10, tolerance = 0.01, max_iterations = 500):
		self.k = k
		self.tolerance = tolerance
		self.max_iterations = max_iterations

	def fit(self, data):

		self.centroids = {}
    
    #initiaate centroids
		for i in range(0, self.k):
			self.centroids[i] = data[i]

		count = self.k

		#start the k cluster algorithm
		for i in range(self.max_iterations):

			self.classes = {}
			for i in range(0, self.k):
				self.classes[i] = []

			distanceArray = []
			distanceArray = [[] for i in range(0, self.k)]
      
			#using the Jaccard distance, classify the features by the min of Jaccard
			for features in data:
				i = 0
				distances = [Jaccard_distance(self.centroids[i], features) for i in range(0, self.k)]
				classification = distances.index(min(distances))
				distanceArray[classification].append(min(distances))
				self.classes[classification].append(features)
       
			previous = dict(self.centroids)

			#recenter the centroids by seeing the 5 most common words, and then attempting to classify them
			for classification in self.classes:
				#print(self.classes[classification])
				initialStuff = fiveCommon(self.classes[classification])
				string1 = ''
				for i in initialStuff:
					string1 = string1 + initialStuff[i] + ' '
				self.centroids[classification] = string1

			isOptimal = True

			for centroid in self.centroids:

				original_centroid = previous[centroid]
				curr = self.centroids[centroid]

				#see if the adjusted centroid is more than tolerated, using Jaccard sitance
				if Jaccard_distance(curr, original_centroid) > self.tolerance:
					isOptimal = False
				
			if isOptimal:
				break

In [0]:
#insert | for text parsing
def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    removal = ''
    for i in test_str:
      if i == '|' and skip1c > 0:
          skip1c -= 1
          removal += i
          return removal
      if i == '|':
          skip1c += 1
          removal+=i
      elif skip1c > 0:
          removal+=i

In [605]:
def main():
  
  with open('/content/bbchealth.txt', 'r') as f:
    x = f.readlines()

  j = 0
  for i in x :

    i = x[j]
    
    
    #remove url
    no_http = i.split('http', 1)[0]
    insert_char = '|' + no_http
    
    #remove id and timestamp
    no_columns = a(insert_char)
    no_id = insert_char.replace(no_columns, '')
    insert_char = '|' + no_id
    no_columns = a(insert_char)
    no_time = insert_char.replace(no_columns, '')


    #this data set did not include @ or #, so I did not have to remove them

    #to lower case and remove the 
    lower_case = no_time.lower()

    #final_string = lower_case.replace("'", '')
    punctuations = '''!-()[]{};:'"\,<'>./?@#$%^&*_~'''

    # remove punctuation from the string
    no_punct = ""
    for char in lower_case:
      if char not in punctuations:
          no_punct = no_punct + char
    x[j] = no_punct
    j+=1

  k = 10
  print(" ---------------------------------------------------- ")
  print("|  Value of K   |   SSE   |   Size of Each Cluster   |")
  print(" ---------------------------------------------------- ")
  for k in range(5, 20, 5):
    km = K_Means(k, 0, 500000)
    km.fit(x)

    distance = 0
    clusterDistance = 0
    i = 0

    for i in km.classes:
      distance = 0
      for centroid in km.centroids:
        for features in km.classes[i]:
          #once again use Jaccard distance to figure out what is the distance
          #thus having a solid cluster distance for the sse
          distance = Jaccard_distance(km.centroids[centroid], features)
          distance = distance * distance
          clusterDistance+=distance

    count = 0
    for i in km.classes:
      print("")
      if count == 0:
          print("|       ", k, "    |  ", round(clusterDistance, 2), " |         ", i, ": ", len(km.classes[i]), "      |")
          count = 1
      else:
          print("|               |         |         ", i, ": ", len(km.classes[i]), "       |")
    print(" ---------------------------------------------------- ")

if __name__ == "__main__":
  main()

 ---------------------------------------------------- 
|  Value of K   |   SSE   |   Size of Each Cluster   |
 ---------------------------------------------------- 

|        5     |   18890.51  |          0 :  2431       |

|               |         |          1 :  403        |

|               |         |          2 :  230        |

|               |         |          3 :  225        |

|               |         |          4 :  640        |
 ---------------------------------------------------- 

|        10     |   37374.2  |          0 :  1496       |

|               |         |          1 :  774        |

|               |         |          2 :  376        |

|               |         |          3 :  718        |

|               |         |          4 :  46        |

|               |         |          5 :  114        |

|               |         |          6 :  131        |

|               |         |          7 :  104        |

|               |         |          8 :  124 